## GRU research
We've researched LSTM and compared it to SARIMAX. WE also saw that GRU was used for resolving our subject. Hence why we would like to compare GRU with LSTM and SARIMAX.

### What is GRU?
GRU stands for Gated Recurrent Unit. It's a version of LSTM. Compared to LSTM, GRU haS some important differences;
- GRU does not have a hidden state.
- Next, the processes of determining what the cell states forgets and what part of the cell state is written to are consolidated into a single gate. Only the portion of the cell state that has been erased is written to.
- Finally, the entire cell state is given as an output. This is different from the LSTM cell which chooses what to read from the cell state to produce an output.

All these differences combined gives GRU a simpler design with less parameters. This could be seen as a pro or a con.
Due to less parameters, memory size and therefore increase in training speed, GRU should be faster than LSTM. However the increase in speed comes with loss in features.

GRU should outperform LSTM when it comes to handling short-distance relations. Thus, GRU should be better to get quick results. However, LSTM would outperform GRU when it comes to long-distance relations. Now, we have to decide how big are data will be and for how long we should train it before we see the performance difference between those two.


## Using GURU

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5081664776962694793,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1947287757212732852
 physical_device_desc: "device: XLA_CPU device"]

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

In [4]:
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

In [5]:
data = pd.read_csv('data/household_power_consumption.txt', sep=';', 
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


## Get the average usage per day
Since data is quite large, we would to first see the average usage per day. Before we can do that, we have to combine Data with Time so we can create a proper datetime type for the table.

In [6]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(data)

In [7]:
X_train = []
y_train = []

In [8]:
y_train = data['Global_active_power']
# y_train.set_index('index')
X_train = data.drop('Global_active_power', axis=1)
# X_train.set_index('index')

In [9]:
# for i in range(len(data)):
#     X_train.append(training_set_scaled[i:i,0])
#     y_train.append(training_set_scaled[i,0])

In [10]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [11]:
# y_train = data['Global_active_power'].reset_index()
# y_train.set_index('index')

In [12]:
# X_train, y_train = np.array(X_train), np.array(y_train)

In [13]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [14]:
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=50, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=1))
# Compiling the RNN
regressorGRU.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
# Fitting to the training set
regressorGRU.fit(X_train,y_train,epochs=50,batch_size=30000)

Epoch 1/50


KeyboardInterrupt: 

In [ ]:
# dataset_total = pd.concat(data["Global_active_power"],axis=0)
# inputs = dataset_total
# inputs = np.reshape(inputs, (-1,1))
inputs  = data[['Global_active_power']]

In [ ]:
# Preparing X_test and predicting the prices
x_test = []
X_test = data.drop('Global_active_power', axis=1)
X_test= np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
GRU_predicted_stock_price = regressorGRU.predict(X_test)
GRU_predicted_stock_price = sc.inverse_transform(GRU_predicted_stock_price)

In [ ]:
# Visualizing the results for GRU
plot_predictions(test_set,GRU_predicted_stock_price)